In [1]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import wandb

In [2]:
data1 = np.load('data1.npy')
lab1 = np.load('lab1.npy')
data0 = np.load('data0.npy')
lab0 = np.load('lab0.npy')
data2 = np.load('data2.npy')
lab2 = np.load('lab2.npy')
total_data=np.concatenate((data0,data1,data2),axis=0)
total_lab=np.concatenate((lab0,lab1,lab2),axis=0)

In [3]:
total_size = len(total_data)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size
train_data = total_data[:train_size]
train_lab=total_lab[:train_size]
val_data = total_data[train_size:train_size + val_size]
val_lab=total_lab[train_size:train_size + val_size]
test_data = total_data[train_size + val_size:]
test_lab=total_lab[train_size + val_size:]


In [4]:
train_data = torch.tensor(train_data, dtype=torch.float32)
train_lab = torch.tensor(train_lab, dtype=torch.long)

val_data = torch.tensor(val_data, dtype=torch.float32)
val_lab = torch.tensor(val_lab, dtype=torch.long)

test_data = torch.tensor(test_data, dtype=torch.float32)
test_lab = torch.tensor(test_lab, dtype=torch.long)

# Add channel dimension (1 for grayscale images)
train_data = train_data.unsqueeze(1)  # Shape becomes [num_samples, 1, height, width]
val_data = val_data.unsqueeze(1)
test_data = test_data.unsqueeze(1)

# Create datasets
train_dataset = TensorDataset(train_data, train_lab)
val_dataset = TensorDataset(val_data, val_lab)
test_dataset = TensorDataset(test_data, test_lab)

# Define data loaders
batch_size = 2048
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Check a single batch
for batch_data, batch_labels in train_loader:
    print("Batch data shape:", batch_data.shape)  # Should be [batch_size, 1, height, width]
    print("Batch labels shape:", batch_labels.shape)  # Should be [batch_size]
    break

Batch data shape: torch.Size([2048, 1, 40, 168])
Batch labels shape: torch.Size([2048])


In [5]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x)

class CNNModel(nn.Module):
    def __init__(self, input_shape):
        super(CNNModel, self).__init__()
        self.block1 = ConvBlock(1, 32)
        self.block2 = ConvBlock(32, 64)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Calculate the flattened size dynamically based on the input shape
        with torch.no_grad():
            dummy_input = torch.zeros(1, *input_shape)  # Batch size of 1 for testing
            flattened_size = self._get_flattened_size(dummy_input)

        self.fc1 = nn.Linear(flattened_size, 128)
        self.fc2 = nn.Linear(128, 1)  # Output a single value for regression

        self.dropout = nn.Dropout(0.5)

    def _get_flattened_size(self, x):
        x = self.pool(self.block1(x))
        x = self.pool(self.block2(x))
        return x.view(-1).size(0)

    def forward(self, x):
        x = self.pool(self.block1(x))
        x = self.pool(self.block2(x))

        # Flatten for fully connected layers
        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)  # Output shape: [batch_size, 1]
        return x


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_shape = (1, 40, 168) 
model = CNNModel(input_shape)
model.load_state_dict(torch.load("regression_model.pth", map_location=device))
model.to(device)
model.eval()


/tmp/ipykernel_13610/2099716114.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("regression_model.pth", map_location=device))


CNNModel(
  (block1): ConvBlock(
    (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (block2): ConvBlock(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=26880, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [7]:
def test_model(model, test_loader):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    predictions = []
    targets = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float()
            outputs = model(images).squeeze(1)  # Flatten outputs to [batch_size]

            predictions.extend(outputs.cpu().numpy())
            targets.extend(labels.cpu().numpy())

    rounded_predictions = np.round(predictions)  # Round predictions to nearest integer

    # Calculate accuracy
    correct_predictions = np.sum(rounded_predictions == targets)
    accuracy = correct_predictions / len(targets) * 100

    print(f"Accuracy: {accuracy:.2f}%")


In [8]:
test_model(model, test_loader)


Accuracy: 7.97%
